In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# ABM households file

In [ ]:
inc_abm_13_2020 = pd.read_csv("T:\\ABM\\release\\ABM\\archive\\version_13.2.2\\input\\2020\\households.csv")
inc_abm_13_2025 = pd.read_csv("T:\\ABM\\release\\ABM\\archive\\version_13.2.2\\input\\2025\\households.csv")
inc_abm_13_2035 = pd.read_csv("T:\\ABM\\release\\ABM\\archive\\version_13.2.2\\input\\2035\\households.csv")
inc_abm_13_2050 = pd.read_csv("T:\\ABM\\release\\ABM\\archive\\version_13.2.2\\input\\2050\\households.csv")

In [ ]:
inc_abm_13_2020['yr'] = 2020
inc_abm_13_2025['yr'] = 2025
inc_abm_13_2035['yr'] = 2035
inc_abm_13_2050['yr'] = 2050

In [ ]:
result = pd.concat([inc_abm_13_2020,inc_abm_13_2025,inc_abm_13_2035,inc_abm_13_2050])

In [ ]:
inc_abm = result[['MGRA', 'HINCCAT1','yr']].copy()

In [ ]:
inc_abm['hh'] = 1

In [ ]:
inc_abm.rename(columns={'HINCCAT1':'income_group_id'}, inplace=True)

In [ ]:
incabm = inc_abm.groupby(['MGRA','income_group_id','yr'])[["hh"]].sum().reset_index()

In [ ]:
mssql_engine = create_engine('mssql+pyodbc://sql2014a8/data_cafe?driver=SQL+Server+Native+Client+11.0')

In [ ]:
mgra_13_sql = '''
SELECT [mgra_13] 
      ,[cocpa_13]
      ,[jurisdiction_2015]
      ,[cicpa_13]
  FROM [data_cafe].[ref].[vi_xref_geography_mgra_13]'''
mgra_13 = pd.read_sql(mgra_13_sql,mssql_engine)

In [ ]:
inc = pd.merge(incabm,mgra_13,left_on='MGRA',right_on='mgra_13',how='left')

In [ ]:
inc.loc[inc.income_group_id==1,'lower_bound'] = 0
inc.loc[inc.income_group_id==1,'upper_bound'] = 29999
inc.loc[inc.income_group_id==2,'lower_bound'] = 30000
inc.loc[inc.income_group_id==2,'upper_bound'] = 59999
inc.loc[inc.income_group_id==3,'lower_bound'] = 60000
inc.loc[inc.income_group_id==3,'upper_bound'] = 99999
inc.loc[inc.income_group_id==4,'lower_bound'] = 100000
inc.loc[inc.income_group_id==4,'upper_bound'] = 149999
inc.loc[inc.income_group_id==5,'lower_bound'] = 150000
inc.loc[inc.income_group_id==5,'upper_bound'] = 349999

In [ ]:
inc['interval_width'] = inc.upper_bound-inc.lower_bound +1

In [ ]:
inc.head()

In [ ]:
# Jurisdiction

In [ ]:
inccitycpa = inc.groupby(['jurisdiction_2015','income_group_id','lower_bound',\
                                   'upper_bound','interval_width','yr']\
                                 )[["hh"]].sum().reset_index()

In [ ]:
inccitycpa.sort_values(by=['jurisdiction_2015','yr','income_group_id'],inplace=True)

In [ ]:
inccitycpa.head(10)

In [ ]:
inccitycpa['cumlativesum'] = inccitycpa.groupby(['jurisdiction_2015','yr'])['hh'].cumsum()

In [ ]:
totalhh = inccitycpa.groupby(['jurisdiction_2015','yr'])['hh'].sum().reset_index()

In [ ]:
totalhh.rename(columns={"hh": "totalhh"},inplace=True)

In [ ]:
totalhh.head()

In [ ]:
inccity= pd.merge(inccitycpa,totalhh,left_on=['jurisdiction_2015','yr'],right_on=['jurisdiction_2015','yr'],how='left')

In [ ]:
inccity['halfhh'] = inccity['totalhh']/2

In [ ]:
inccity['halfhh'] = inccity['halfhh'].round() 

In [ ]:
halfhh_firstrow = inccity.loc[inccity.cumlativesum>inccity.halfhh]

In [ ]:
mediancitycpa = halfhh_firstrow.groupby(['jurisdiction_2015','yr']).first()

In [ ]:
mediancitycpa.head()

In [ ]:
mediancitycpa['median_income'] = mediancitycpa.lower_bound +\
((mediancitycpa.halfhh - (mediancitycpa.cumlativesum - mediancitycpa.hh))\
                                                            /mediancitycpa.hh)*mediancitycpa.interval_width

In [ ]:
mediancitycpa.median_income = mediancitycpa.median_income.round() 

In [ ]:
jurisdictions_names_sql = '''
    SELECT zone as jurisdiction_id_data_cafe, name as jurisdiction
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 150'''
jur_name_df = pd.read_sql(jurisdictions_names_sql, mssql_engine)

In [ ]:
mediancitycpa.reset_index(inplace=True)

In [ ]:
mediancitycpa.head()

In [ ]:
jur_name_df.head()

In [ ]:
citymed = pd.merge(mediancitycpa[['jurisdiction_2015','yr','median_income']],\
                   jur_name_df,right_on = 'jurisdiction_id_data_cafe',left_on='jurisdiction_2015')

In [ ]:
citymed.sort_values(by=['jurisdiction_2015','yr'],inplace=True)

In [ ]:
citymed.head()

In [ ]:
# citymed[['cicpa_13','yr','median_income','cicpa']]

In [ ]:
citymed[['jurisdiction_2015','yr','median_income','jurisdiction']].to_csv('median_income_jur.csv')

In [ ]:
# year 2050 only
# city2050 = citymed[['cicpa_13','yr','median_income','cicpa']].loc[citymed.yr==2050].sort_values(by='median_income')

In [ ]:
max_median = city2050.groupby(['cicpa_13','cicpa'])[["median_income"]].max().reset_index().sort_values(by='median_income')

In [ ]:
max_median.median_income.median()

In [ ]:
# one scale to plot
max_median.loc[max_median.median_income < max_median.median_income.median()]

In [ ]:
# other scale for plot
max_median.loc[max_median.median_income >= max_median.median_income.median()]